# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

In [ ]:
import pandas as pd
import numpy as np
import math
import json
import matplotlib.pyplot as plt
import seaborn as sns

In [31]:
# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

## Data understanding

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

In [2]:
portfolio

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7
5,3,"[web, email, mobile, social]",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2
6,2,"[web, email, mobile, social]",10,10,discount,fafdcd668e3743c1bb461111dcafc2a4
7,0,"[email, mobile, social]",0,3,informational,5a8bc65990b245e5a138643cd4eb9837
8,5,"[web, email, mobile, social]",5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d
9,2,"[web, email, mobile]",10,7,discount,2906b810c7d4411798c6938adc9daaa5


In [3]:
portfolio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   reward      10 non-null     int64 
 1   channels    10 non-null     object
 2   difficulty  10 non-null     int64 
 3   duration    10 non-null     int64 
 4   offer_type  10 non-null     object
 5   id          10 non-null     object
dtypes: int64(3), object(3)
memory usage: 612.0+ bytes


In [4]:
portfolio.describe(include='all').round(1)

,reward,channels,difficulty,duration,offer_type,id
count,10.0,10,10.0,10.0,10,10
unique,NaN,4,NaN,NaN,3,10
top,NaN,"[web, email, mobile, social]",NaN,NaN,bogo,ae264e3637204a6fb9bb56bc8210ddfd
freq,NaN,4,NaN,NaN,4,1
mean,4.2,NaN,7.7,6.5,NaN,NaN
std,3.6,NaN,5.8,2.3,NaN,NaN
min,0.0,NaN,0.0,3.0,NaN,NaN
25%,2.0,NaN,5.0,5.0,NaN,NaN
50%,4.0,NaN,8.5,7.0,NaN,NaN
75%,5.0,NaN,10.0,7.0,NaN,NaN


In [24]:
portfolio['channels'].explode().unique()

array(['email', 'mobile', 'social', 'web'], dtype=object)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

In [5]:
profile.head()

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


In [6]:
profile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            14825 non-null  object 
 1   age               17000 non-null  int64  
 2   id                17000 non-null  object 
 3   became_member_on  17000 non-null  int64  
 4   income            14825 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 664.2+ KB


In [ ]:
profile.describe(include='all')

,gender,age,id,became_member_on,income
count,14825,17000.000000,17000,1.700000e+04,14825.000000
unique,3,NaN,17000,NaN,NaN
top,M,NaN,e4052622e5ba45a8b96b59aba68cf068,NaN,NaN
freq,8484,NaN,1,NaN,NaN
mean,NaN,62.531412,NaN,2.016703e+07,65404.991568
std,NaN,26.738580,NaN,1.167750e+04,21598.299410
min,NaN,18.000000,NaN,2.013073e+07,30000.000000
25%,NaN,45.000000,NaN,2.016053e+07,49000.000000
50%,NaN,58.000000,NaN,2.017080e+07,64000.000000
75%,NaN,73.000000,NaN,2.017123e+07,80000.000000


In [26]:
profile['gender'].value_counts(dropna=False)

gender
M       8484
F       6129
None    2175
O        212
Name: count, dtype: int64

In [27]:
profile[profile['age'] >= 100]

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN
6,None,118,8ec6ce2a7e7949b1bf142def7d0e0586,20170925,NaN
7,None,118,68617ca6246f4fbc85e91a2a49552598,20171002,NaN
...,...,...,...,...,...
16980,None,118,5c686d09ca4d475a8f750f2ba07e0440,20160901,NaN
16982,None,118,d9ca82f550ac4ee58b6299cf1e5c824a,20160415,NaN
16989,None,118,ca45ee1883624304bac1e4c8a114f045,20180305,NaN
16991,None,118,a9a20fa8b5504360beb4e7c8712f8306,20160116,NaN


In [29]:
profile[profile['age'] < 18]

,gender,age,id,became_member_on,income


In [30]:
len(profile[profile['age'] >= 100])/len(profile)*100

12.894117647058822

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

In [8]:
transcript.head()

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


In [10]:
transcript.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306534 entries, 0 to 306533
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   person  306534 non-null  object
 1   event   306534 non-null  object
 2   value   306534 non-null  object
 3   time    306534 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 9.4+ MB


In [9]:
transcript.describe(include='all')

,person,event,value,time
count,306534,306534,306534,306534.000000
unique,17000,4,5121,NaN
top,94de646f7b6041228ca7dec82adb97d2,transaction,{'offer id': '2298d6c36e964ae4a3e7e9706d1fb8c2'},NaN
freq,51,138953,14983,NaN
mean,NaN,NaN,NaN,366.382940
std,NaN,NaN,NaN,200.326314
min,NaN,NaN,NaN,0.000000
25%,NaN,NaN,NaN,186.000000
50%,NaN,NaN,NaN,408.000000
75%,NaN,NaN,NaN,528.000000


In [11]:
#converted = pd.Series([0 if 'offer id' in x else 1 for x in transcript['value']])
amount = pd.Series([x['amount'] if 'amount' in x else 0 for x in transcript['value']])
offer_id = pd.Series([x['offer id'] if 'offer id' in x else 0 for x in transcript['value']])

#converted.name = 'converted'
amount.name = 'amount'
offer_id.name = 'offer_id'

In [12]:
transcript_b = pd.concat([
    transcript.drop(columns=['value']),
    offer_id,
    #converted,
    amount
], axis=1)

In [13]:
transcript

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0
...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,transaction,{'amount': 1.5899999999999999},714
306530,68213b08d99a4ae1b0dcb72aebd9aa35,transaction,{'amount': 9.53},714
306531,a00058cf10334a308c68e7631c529907,transaction,{'amount': 3.61},714
306532,76ddbd6576844afe811f1a3c0fbb5bec,transaction,{'amount': 3.5300000000000002},714


In [14]:
transcript_b

,person,event,time,offer_id,amount
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.00
1,a03223e636434f42ac4c3df47e8bac43,offer received,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,0.00
2,e2127556f4f64592b11af22de27a7932,offer received,0,2906b810c7d4411798c6938adc9daaa5,0.00
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,0,fafdcd668e3743c1bb461111dcafc2a4,0.00
4,68617ca6246f4fbc85e91a2a49552598,offer received,0,4d5c57ea9a6940dd891ad53e9dbe8da0,0.00
...,...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,transaction,714,0,1.59
306530,68213b08d99a4ae1b0dcb72aebd9aa35,transaction,714,0,9.53
306531,a00058cf10334a308c68e7631c529907,transaction,714,0,3.61
306532,76ddbd6576844afe811f1a3c0fbb5bec,transaction,714,0,3.53


In [15]:
transcript_b.groupby(['person', 'event', 'offer_id']).count().unstack().reset_index()


person            event  time  \
offer_id                                                        0   
0         0009655768c64bdeb2e877511632db8f  offer completed   3.0   
1         0009655768c64bdeb2e877511632db8f   offer received   NaN   
2         0009655768c64bdeb2e877511632db8f     offer viewed   NaN   
3         0009655768c64bdeb2e877511632db8f      transaction   8.0   
4         00116118485d4dfda04fdbaba9a87b5c   offer received   NaN   
...                                    ...              ...   ...   
63175     fffad4f4828548d1b5583907f2e9906b      transaction  12.0   
63176     ffff82501cea40309d5fdd7edcca4a07  offer completed   6.0   
63177     ffff82501cea40309d5fdd7edcca4a07   offer received   NaN   
63178     ffff82501cea40309d5fdd7edcca4a07     offer viewed   NaN   
63179     ffff82501cea40309d5fdd7edcca4a07      transaction  15.0   

                                                                            \
offer_id 0b1e1539f2cc45b7b9fa7c272da2e1d7 2298d6c36e964ae4a3e7e9706d1fb8c2   
0                                     NaN                              NaN   
1                                     NaN                              NaN   
2                                     NaN                              NaN   
3                                     NaN                              NaN   
4                                     NaN                              NaN   
...                                   ...                              ...   
63175                                 NaN                              NaN   
63176                                 NaN                              NaN   
63177                                 1.0                              NaN   
63178                                 1.0                              NaN   
63179                                 NaN                              NaN   

                                                                            \
offer_id 2906b810c7d4411798c6938adc9daaa5 3f207df678b143eea3cee63160fa8bed   
0                                     NaN                              NaN   
1                                     1.0                              1.0   
2                                     NaN                              1.0   
3                                     NaN                              NaN   
4                                     NaN                              NaN   
...                                   ...                              ...   
63175                                 NaN                              NaN   
63176                                 NaN                              NaN   
63177                                 3.0                              NaN   
63178                                 3.0                              NaN   
63179                                 NaN                              NaN   

                                                                            \
offer_id 4d5c57ea9a6940dd891ad53e9dbe8da0 5a8bc65990b245e5a138643cd4eb9837   
0                                     NaN                              NaN   
1                                     NaN                              1.0   
2                                     NaN                              1.0   
3                                     NaN                              NaN   
4                                     NaN                              NaN   
...                                   ...                              ...   
63175                                 NaN                              NaN   
63176                                 NaN                              NaN   
63177                                 NaN                              NaN   
63178                                 NaN                              NaN   
63179                                 NaN                              NaN   

                                           ...  \
offer_id 9b98b8c7a33c4b65b9aebfe6a799e6d9  

In [16]:
transcript_b.groupby(['person'])[['offer_id', 'event']].count() #.unstack().fillna(0) #.reset_index(drop=True)


,offer_id,event
person,,
0009655768c64bdeb2e877511632db8f,20,20
00116118485d4dfda04fdbaba9a87b5c,7,7
0011e0d4e6b944f998e987f904e8c1e5,18,18
0020c2b971eb4e9188eac86d93036a77,19,19
0020ccbbb6d84e358d3414a3ff76cffd,23,23
...,...,...
fff3ba4757bd42088c044ca26d73817a,23,23
fff7576017104bcc8677a8d63322b5e1,18,18
fff8957ea8b240a6b5e634b6ee8eafcf,10,10


In [17]:
transcript.event.value_counts()

event
transaction        138953
offer received      76277
offer viewed        57725
offer completed     33579
Name: count, dtype: int64

In [18]:
transcript_transaction = transcript_b[transcript_b['event'] == 'transaction'].drop(columns='event')
transcript_received = transcript_b[transcript_b['event'] == 'offer received'].drop(columns='event')
transcript_viewed = transcript_b[transcript_b['event'] == 'offer viewed'].drop(columns='event')
transcript_completed = transcript_b[transcript_b['event'] == 'offer completed'].drop(columns='event')

#transcript_b_merge = pd.merge(how='left', left=transcript_b_received, right=transcript_b_transaction, on=['person', 'person'])

In [19]:
transcript_received

,person,time,offer_id,amount
0,78afa995795e4d85b5d9ceeca43f5fef,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0
1,a03223e636434f42ac4c3df47e8bac43,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,0.0
2,e2127556f4f64592b11af22de27a7932,0,2906b810c7d4411798c6938adc9daaa5,0.0
3,8ec6ce2a7e7949b1bf142def7d0e0586,0,fafdcd668e3743c1bb461111dcafc2a4,0.0
4,68617ca6246f4fbc85e91a2a49552598,0,4d5c57ea9a6940dd891ad53e9dbe8da0,0.0
...,...,...,...,...
257882,d087c473b4d247ccb0abfef59ba12b0e,576,ae264e3637204a6fb9bb56bc8210ddfd,0.0
257883,cb23b66c56f64b109d673d5e56574529,576,2906b810c7d4411798c6938adc9daaa5,0.0
257884,6d5f3a774f3d4714ab0c092238f3a1d7,576,2298d6c36e964ae4a3e7e9706d1fb8c2,0.0
257885,9dc1421481194dcd9400aec7c9ae6366,576,ae264e3637204a6fb9bb56bc8210ddfd,0.0


In [20]:
transcript_transaction

,person,time,offer_id,amount
12654,02c083884c7d45b39cc68e1314fec56c,0,0,0.83
12657,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,0,0,34.56
12659,54890f68699049c2a04d415abc25e717,0,0,13.23
12670,b2f1cd155b864803ad8334cdf13c4bd2,0,0,19.51
12671,fe97aa22dd3e48c8b143116a8403dd52,0,0,18.97
...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,714,0,1.59
306530,68213b08d99a4ae1b0dcb72aebd9aa35,714,0,9.53
306531,a00058cf10334a308c68e7631c529907,714,0,3.61
306532,76ddbd6576844afe811f1a3c0fbb5bec,714,0,3.53


In [21]:
transcript_viewed

,person,time,offer_id,amount
12650,389bc3fa690240e798340f5a15918d5c,0,f19421c1d4aa40978ebb69ca19b0e20d,0.0
12651,d1ede868e29245ea91818a903fec04c6,0,5a8bc65990b245e5a138643cd4eb9837,0.0
12652,102e9454054946fda62242d2e176fdce,0,4d5c57ea9a6940dd891ad53e9dbe8da0,0.0
12653,02c083884c7d45b39cc68e1314fec56c,0,ae264e3637204a6fb9bb56bc8210ddfd,0.0
12655,be8a5d1981a2458d90b255ddc7e0d174,0,5a8bc65990b245e5a138643cd4eb9837,0.0
...,...,...,...,...
306441,d56386cf344c4829bbf420d1895dca37,714,5a8bc65990b245e5a138643cd4eb9837,0.0
306450,9b51e8797290403b90d09d864dec4b94,714,3f207df678b143eea3cee63160fa8bed,0.0
306483,84fb57a7fe8045a8bf6236738ee73a0f,714,5a8bc65990b245e5a138643cd4eb9837,0.0
306490,abc4359eb34e4e2ca2349da2ddf771b6,714,3f207df678b143eea3cee63160fa8bed,0.0


In [22]:
transcript_completed

,person,time,offer_id,amount
12658,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,0,0,0.0
12672,fe97aa22dd3e48c8b143116a8403dd52,0,0,0.0
12679,629fc02d56414d91bca360decdfa9288,0,0,0.0
12692,676506bad68e4161b9bbaffeb039626b,0,0,0.0
12697,8f7dd3b2afe14c078eb4f6e6fe4ba97d,0,0,0.0
...,...,...,...,...
306475,0c027f5f34dd4b9eba0a25785c611273,714,0,0.0
306497,a6f84f4e976f44508c358cc9aba6d2b3,714,0,0.0
306506,b895c57e8cd047a8872ce02aa54759d6,714,0,0.0
306509,8431c16f8e1d440880db371a68f82dd0,714,0,0.0
